# Assignment 5 - Data Visualization using # Pymaceuticals Inc.
---




### Analysis

     * When looking at the box plot the the effect of capomulin and Ramicane are relatelively same but defintley 
       more efficient than infubinol and Ceftamin.
     
     * When looking at one particular mouse with Capomulin drug it's significantly responding well after 30 days.
     
     * We have only one outlier for the drug infubinol while other drugs have no outliers thus the other drugs are 
       consistant in this study.
     
     * When we comparing Mouse weight and Tumor Volume the mouse with more weihgt have more tumor volume. It shows that 
       mouse with more weight is not responding well for the drugs. 
     
     * The correlation between mouse weight and the average tumor volume is:0.84. It's evident that we have a strong 
       correlation so the relationship between weight and tumor volume is positively relative. 

In [25]:
# Prepare the Data
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [26]:
# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path ="data/study_results.csv"

In [27]:
# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

In [46]:
# Combine the data into a single DataFrame
combined_df = pd.merge(mouse_metadata,study_results, how="outer", on ="Mouse ID")
# Display the data table for preview
combined_df.head()

In [47]:
# Checking the number of mice.
no_of_mice = len(combined_df["Mouse ID"].value_counts())
no_of_mice

In [48]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# combined_df["Mouse ID"].duplicated()
dup_mice_df = combined_df.loc[combined_df.duplicated(subset=['Mouse ID','Timepoint']),'Mouse ID'].unique()
dup_mice_df

In [49]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mice_df = combined_df.loc[combined_df['Mouse ID']=="g989"]
duplicate_mice_df

In [50]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = combined_df.loc[combined_df['Mouse ID']!="g989"]
# combined_df.loc[~combined_df['Mouse ID'].isin(['g989','g990'])]
# clean_df.isna().any()
clean_df

In [51]:
# Checking the number of mice in the clean DataFrame.
mice_number_df =len( clean_df['Mouse ID'].unique())
mice_number_df

# Summary Statistics

In [52]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 

mean_vol = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
median_vol = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
var_vol = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].var()
std_vol = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
sem_vol = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].sem()

# Assemble the resulting series into a single summary DataFrame.
sum_df = pd.DataFrame({"Mean Tumor Volume": mean_vol,
                       "Median Tumor Volume": median_vol,
                       "Variance Tumor Volume":var_vol,
                       "STD Tumor Volume": std_vol,
                       "SEM Tumor Volume": sem_vol                     
                                 
    
    
})
sum_df

In [53]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
sum_df2 = clean_df.groupby('Drug Regimen').agg({'Tumor Volume (mm3)': ['mean','median','var','std','sem']})
sum_df2

  ## Bar & Pie Charts

In [56]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.
drug_count = clean_df['Drug Regimen'].value_counts()
drug_count.plot(kind="bar", ylabel="# of observed Time Points")
plt.show()

In [58]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.
plt.bar(x=drug_count.index, height=drug_count.values)
plt.xticks(rotation=90)
plt.ylabel("# of observed Time Points")
plt.xlabel("Drug Regimen")
plt.show()

In [59]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
gender_count = clean_df['Sex'].value_counts()
explode = (0.01, 0)
gender_count.plot(kind="pie", ylabel="Sex",autopct='%.1f%%', explode = explode)
plt.show()

In [61]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
explode = (0.01, 0)
plt.pie(gender_count.values, labels=gender_count.index, autopct='%.1f%%',explode = explode)
plt.ylabel("Sex")
plt.show()

## Quartiles, Outliers and Boxplots### 

In [63]:
#A DatFrame that has the last timepoint for each mouse ID is created using groupby.
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
greatest_tp_df = pd.DataFrame(clean_df.groupby('Mouse ID')['Timepoint'].max().sort_values())

# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
merged_df = pd.merge(clean_df, greatest_tp_df, on=['Mouse ID','Timepoint']).rename(columns={"Timepoint_x": "Timepoint", "Timepoint_y": "Max Timepoint"})

merged_df.head()



In [68]:
# # Put treatments into a list for for loop (and later for plot labels)
# drugs =['Capomulin', 'Ramicane', 'Infubinol','Ceftamin']

# # Create empty list to fill with tumor vol data (for plotting)
tumor_vol = []

# # Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
#     # Locate the rows which contain mice on each drug and get the tumor volumes
for drug in drugs:
    df1=merged_df.loc[merged_df['Drug Regimen']== drug,'Tumor Volume (mm3)']
        
    # add subset 
    tumor_vol.append(df1)
    
     # Calculate the IQR and quantitatively determine if there are any potential outliers.
     # Determine outliers using upper and lower bounds

    quartiles = df1.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq

    print(f'IQR for {drug} is : {iqr}')
     # Find upper and lower bounds to help identify outliers for each regimen
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
    print(f'Lower Bound for {drug}: {lower_bound}')
    print(f'Upper Bound for {drug}: {upper_bound}')
    
     #check for outliers
    df_outliers = df1[(df1 < lower_bound) | (df1> upper_bound)].count()
    print(f'Number of {drug} Outlier: {df_outliers}')

In [73]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.
plt.boxplot(tumor_vol)
plt.title('Final Tumor Volume by Drug')
plt.ylabel('Final Tumor Volume (mm3)')
plt.xticks([1, 2, 3, 4], ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin'])
plt.show()

### Line and Scatter Plots

In [75]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin
singlemouse_Capomulin = clean_df.loc[clean_df['Mouse ID']=="l509",:]
x_axis = singlemouse_Capomulin['Timepoint'].count()
x= singlemouse_Capomulin['Timepoint']
y= singlemouse_Capomulin['Tumor Volume (mm3)']
plt.plot(x,y,color = "blue")
plt.title(" Capomulin Treatment of Mouse ID: l509")
plt.xlabel("Timepoint (days)")
plt.ylabel("Tumor Volume (mm3)")
plt.show()

In [78]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen
# Isolate cases that only use the Capomulin regimen

capomulin_df = clean_df.loc[clean_df['Drug Regimen'] == 'Capomulin']

# Find average tumor volume for each mouse
avg_vol_df = pd.DataFrame(capomulin_df.groupby('Mouse ID')['Tumor Volume (mm3)'].mean().sort_values())
avg1_vol_df=avg_vol_df.rename(columns={"Tumor Volume (mm3)":"Ave Tumor Volume (mm3)"}).reset_index()
avg1_vol_df

# Merge average tumor volume onto data_df and drop duplicates
avg_vol_df = pd.merge(capomulin_df, avg1_vol_df , on='Mouse ID')
final_avg_vol_df = avg_vol_df [['Weight (g)', 'Ave Tumor Volume (mm3)']].drop_duplicates()
final_avg_vol_df

# Plot the scatter plot with final ave weight with Ave Tumor Volume
x = final_avg_vol_df['Weight (g)']
y = final_avg_vol_df['Ave Tumor Volume (mm3)']
plt.scatter(x, y)
plt.xlabel("weight(g)")
plt.ylabel("Ave Tumor Volume (mm3))")
plt.title("Mouse Weight vs Tumor Volume (mm3)")
plt.show()

### Correlation and Regression

In [81]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
#Take x,y value from the above part

x = final_avg_vol_df['Weight (g)']
y = final_avg_vol_df['Ave Tumor Volume (mm3)']
# Calculate the correlation coefficient between mouse weight and average tumor volume
correlation = st.pearsonr(x,y)
print(f'The correlation between mouse weight and the average tumor volume is:{round(correlation[0],2)}')

# Calculate linear regression
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x, y)
regress_values = x * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot linear regression on top of scatter plot      
plt.scatter(x,y)
plt.plot(x,regress_values,"r-")
plt.annotate(line_eq,(20,40),fontsize=15,color="red")
plt.xlabel("weight(g)")
plt.ylabel("Ave Tumor Volume (mm3))")
plt.title("Mouse Weight vs Tumor Volume (mm3)")
plt.show()